# Tmax States Final

A notebook to subset Tmax daily for the 13000 GHS urban areas to identify dates >40c, consecuritve days >40 c etc.

Moved from cpt_tmax_old on 2020.02.22 by Cascade Tuholske

#### Dependencies

In [156]:
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
import glob
from statistics import mean
import julian
import time 
import multiprocessing as mp 
from multiprocessing import Pool
import os

# Functions

#### Step 1 - Function Loads all Tmax Data as an X-array

In [157]:
def read_data(dir_path, space_dim, time_dim):
    """ Function reads in all Tmax .csv files, joins them by date along the x-axis
    and returns the whole record as a x-array data array
    
    Args:   
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    fn_list = sorted(glob.glob(dir_path+'*.csv'))
    df_out = pd.DataFrame()
    date_list = []

    # Open all Tmax files and concat into a df
    for i, fn in enumerate(fn_list):    
        # Open the CSV
        df = pd.read_csv(fn)

        # Get the city ids 
        if i == 1:
            df_id = df[space_dim]

        # get only the Tmax columns and concate date list 
        df_temp = df.iloc[:,3:] # get only temp columns
        date_list = date_list+list(df_temp.columns)

        # Drop cities w/ no temp record 
        df_temp_drop = df_temp.dropna()

        # Merge
        df_out = pd.concat([df_out, df_temp_drop], axis=1)
        print(df_out.shape)
    
    # make date into an array
    tmax_arr = df_out.to_numpy()

    # Make data into an xr.DataArray
    tmax_xr_da = xr.DataArray(tmax_arr, coords=[df_id, date_list], 
                             dims=[space_dim, time_dim])
    return tmax_xr_da

#### Step 2 Function finds all the Tmax Events and writes it to a dateframe w/ dates for each city

In [158]:
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = float of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # #CPT 2020.02.23 
        # dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    # df_out['total_days'] = dayTot_list #CPT 2020.02.23
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out

#### Step 3 Function chunks Tmax events

In [159]:
# # Let's split test 2
# def chunk_df(df, n, DATA_IN):
#     "Function chunks a data frame and puts chunks in a list, n is number of chunks"
#     list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
    
#     for i in range(n):
#         out_fn = DATA_IN+'chunk_'+str(i)+'.csv'
#         list_df[i].to_csv(out_fn)
    

#### Step 4 Function splits the dataset into Tmax events (continuous days >Tmax) for each city

In [160]:
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

def event_split(dates, ID_HDC_G0, intensity, tmax): #, total_days): #CPT 2020.02.23
    
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city

    """

    # city id
    city_id = ID_HDC_G0
    # tot_days = total_days #CPT 2020.02.23
    
    # lists to fill
    city_id_list = []
    # tot_days_list = [] #CPT 2020.02.23
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    year_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = intense.sum() # total intensity during event
        
        start = counter # reset start to current end (e.g. counter)
        year = event_dates[0].split('.')[0]
        
        # fill lists
        city_id_list.append(city_id)
        year_list.append(year)
        # tot_days_list.append(tot_days) #CPT 2020.02.23
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['year'] = year_list
    # df_out['total_days'] = tot_days_list #CPT 2020.02.23
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

#### Step 4 function feeds output from function 3 into function 4

In [177]:
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats

        NOTE - If you add arguments to event_split to make more states,
        be sure to update this function

        args:
            df: input dataframe
    """
    df_out = pd.DataFrame()


    # NOTE - If you add arguments to event_split to make more stats,
    # be sure to update this function

    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        # total_days = row['total_days'] # get total number of tmax days -- CPT 2020.02.23

        df = event_split(dates, ID_HDC_G0, intensity, tmax)# , total_days) #CPT 2020.02.23

        df_out = df_out.append(df)

    return df_out
    
    

#### Step 5 function threads it all together

In [178]:
def run_stats(dir_path, space_dim, time_dim, Tthresh, fn_out):
    
    """ Function ties all the Tmax Stats functions together and writes final stats for each Tmax 
    event to a .csv file. Returns results as a dataframe if needed
    
    Args:
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
        Tthresh = float of temp threshold
        fn_out = file and path to write final csv
        
    """
    
    # read in data
    step1= read_data(dir_path, space_dim = space_dim, time_dim = time_dim)
    #step1_sub = step1[:,:10] # subset data for testing
    print('Stack x-array made')
    
    # Mask data based on Tmax threshold ... we're using 40.6C
    step2 = tmax_days(step1, Tthresh)
    print('Tmax masked')
    
    
    # Calculate stats
    step3 = tmax_stats(step2)
    print('Stats made')

    # Save file out
    #step3.to_csv(fn_out)
    
    return step3

    print('done')

# Run this shit

In [179]:
DATA_IN = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-HI/' # output from avg temp
DATA_OUT = '/home/cascade/projects/UrbanHeat/data/processed/'

In [180]:
dir_path = DATA_IN 
space_dim = 'ID_HDC_G0'
time_dim = 'date'
Tthresh = 40.6
fn_out = DATA_OUT+'All_data_HI406.csv'

In [181]:
runit = run_stats(dir_path, space_dim, time_dim, Tthresh, fn_out)

(13067, 365)
(13067, 731)
(13067, 1096)
(13067, 1461)
(13067, 1826)
(13067, 2192)
(13067, 2557)
(13067, 2922)
(13067, 3287)
(13067, 3653)
(13067, 4018)
(13067, 4383)
(13067, 4748)
(13067, 5114)
(13067, 5479)
(13067, 5844)
(13067, 6209)
(13067, 6575)
(13067, 6940)
(13067, 7305)
(13067, 7670)
(13067, 8036)
(13067, 8401)
(13067, 8766)
(13067, 9131)
(13067, 9497)
(13067, 9862)
(13067, 10227)
(13067, 10592)
(13067, 10958)
(13067, 11323)
(13067, 11688)
(13067, 12053)
(13067, 12419)
Stack x-array made
Tmax masked
Stats made


In [175]:
len(runit)

1521

In [ ]:
step1= read_data(dir_path, space_dim = space_dim, time_dim = time_dim)

In [164]:
step1_sub = step1[:,:10]

In [143]:
Tthresh = 40.6
step2 = tmax_days(step1_sub, Tthresh)

In [154]:
df_out = tmax_stats(step2)

### NOTE 2020.02.22 --- see if this write out correctly
#df_out.to_csv('/home/cascade/projects/UrbanHeat/data/All_stats_HI406_test.csv')

In [155]:
df_out.head()

,ID_HDC_G0,year,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax
0,10630,1983,2,41.169058,0.569058,1.138117,"[1983.01.06, 1983.01.07]","[0.29257217264085256, 0.8455443555204525]","[40.892572172640854, 41.445544355520454]"
0,12799,1983,3,42.726401,2.126401,6.379204,"[1983.01.01, 1983.01.02, 1983.01.03]","[2.7539840539556195, 2.8988528219620875, 0.726...","[43.35398405395562, 43.49885282196209, 41.3263..."
1,12799,1983,1,42.101542,1.501542,1.501542,[1983.01.07],[1.5015423810544917],[42.10154238105449]
2,12799,1983,2,41.282618,0.682618,1.365235,"[1983.01.09, 1983.01.10]","[1.073923854489017, 0.2913114130661114]","[41.67392385448902, 40.89131141306611]"
0,12820,1983,2,41.971993,1.371993,2.743987,"[1983.01.01, 1983.01.02]","[1.2265718525154057, 1.5174147668343707]","[41.82657185251541, 42.11741476683437]"


# Check Dec 31 - Jan 1 Events

In [87]:
stats_in = pd.read_csv('/home/cascade/projects/UrbanHeat/data/All_stats_HI406_test.csv')

In [89]:
len(stats_in)

5668643

In [97]:
stats_in.head()

,Unnamed: 0,ID_HDC_G0,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax
0,0,3554,3,1,43.438784,2.838784,2.838784,['1989.06.30'],[2.83878425],[43.43878425]
1,1,3554,3,1,41.557437,0.957437,0.957437,['2003.07.29'],[0.95743653],[41.55743653]
2,2,3554,3,1,43.966580,3.366580,3.366580,['2010.07.07'],[3.36658027],[43.96658027]
3,0,2429,9,1,40.795059,0.195059,0.195059,['1984.07.31'],[0.19505924],[40.79505924]
4,1,2429,9,1,42.365319,1.765319,1.765319,['1991.07.08'],[1.76531907],[42.36531907]


In [101]:
def date_search(df, date):
    
    """ Searches Tmax data frame to find dates within a Tmax event with the goal of finding 12.31-01.01 overlap
    Args:
        df = tmax df
        data = date you want to find
    
    Returns df with event id, event dates, city id, year, and tmax temps 
    """

    event_id_list = []
    event_dates_list = []
    city_id_list = []
    event_year_list = []
    tmax_list = []
    total_days_list = []
    
    for index, row in df.iterrows():
        if date in row['event_dates']:
            
            #event_id = row['Event_ID']
            event_dates = row['event_dates']
            city_id = row['ID_HDC_G0']
            #event_year = row['year']
            tmax = row['tmax']
            total_days = row['total_days']
            
            #event_id_list.append(event_id)
            event_dates_list.append(event_dates)
            city_id_list.append(city_id)
            #event_year_list.append(event_year)
            tmax_list.append(tmax)
            total_days_list.append(total_days)
    
    df_out = pd.DataFrame()
    df_out['ID_HDC_G0'] = city_id_list
    #df_out['Event_ID'] = event_id_list
    df_out['tmax'] = tmax_list
    df_out['event_dates'] = event_dates_list
    #df_out['year'] = event_year_list
    df_out['total_days'] = total_days_list
    
    return df_out

In [102]:
date = '12.31'

In [103]:
es = date_search(stats_in, date)

In [112]:
es.head(10)

,ID_HDC_G0,tmax,event_dates,total_days
0,350,[41.88869451],['2010.12.31'],5290
1,375,[41.73767473 41.40027191],['1989.12.30' '1989.12.31'],6124
2,375,[43.101553],['2002.12.31'],6124
3,375,[41.77394773],['2005.12.31'],6124
4,375,[44.34242563],['2007.12.31'],6124
5,375,[43.78522023 41.19842822],['2010.12.31' '2011.01.01'],6124
6,375,[41.25341645 44.49574652 43.60375475 43.491900...,['2015.12.28' '2015.12.29' '2015.12.30' '2015....,6124
7,390,[41.59291018],['2007.12.31'],4278
8,391,[41.03117554],['2007.12.31'],3495
9,7728,[41.53073374],['2012.12.31'],4364


In [107]:
es['event_dates'][6]

"['2015.12.28' '2015.12.29' '2015.12.30' '2015.12.31' '2016.01.01']"

In [129]:
stats_in[stats_in['ID_HDC_G0'] == 375]

,Unnamed: 0,ID_HDC_G0,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax
2441868,0,375,6124,3,42.703217,2.103217,6.309651,['1983.01.20' '1983.01.21' '1983.01.22'],[1.8220826 2.83717738 1.65039061],[42.4220826 43.43717738 42.25039061]
2441869,1,375,6124,1,41.869750,1.269750,1.269750,['1983.01.30'],[1.26975034],[41.86975034]
2441870,2,375,6124,2,41.852571,1.252571,2.505142,['1983.02.01' '1983.02.02'],[1.88940372 0.61573789],[42.48940372 41.21573789]
2441871,3,375,6124,1,45.847263,5.247263,5.247263,['1983.02.06'],[5.24726308],[45.84726308]
2441872,4,375,6124,4,45.447617,4.847617,19.390470,['1983.03.05' '1983.03.06' '1983.03.07' '1983....,[5.68699648 6.94049717 5.77288793 0.99008814],[46.28699648 47.54049717 46.37288793 41.59008814]
...,...,...,...,...,...,...,...,...,...,...
2442959,1091,375,6124,2,42.267758,1.667758,3.335516,['2016.11.30' '2016.12.01'],[1.16794035 2.16757582],[41.76794035 42.76757582]
2442960,1092,375,6124,6,44.752279,4.152279,24.913672,['2016.12.03' '2016.12.04' '2016.12.05' '2016....,[0.68738888 0.1823002 7.6203832 6.98349534 6...,[41.28738888 40.7823002 48.2203832 47.583495...
2442961,1093,375,6124,3,41.693444,1.093444,3.280332,['2016.12.12' '2016.12.13' '2016.12.14'],[0.31888825 1.60105008 1.36039368],[40.91888825 42.20105008 41.96039368]
2442962,1094,375,6124,2,42.878464,2.278464,4.556928,['2016.12.18' '2016.12.19'],[2.82346886 1.73345885],[43.42346886 42.33345885]


In [134]:
int(stats_in['event_dates'][2441868].split(' ')[0].split('[\'')[1].split('.')[0])

1983

# Old code

#### UPDATE

In [70]:
def stats_parallel(fn):
    
    """ EXPLAIN
    """

    # print current process
    print(mp.current_process())
    
    chunk = fn.split('chunk_')[1].split('.csv')[0] # Updated 2020.02.19 CPT
    df_days = pd.read_csv(fn)

    # Open the GHS-ID List with GeoPANDAS read_file
    ghs_ids_fn = 'GHS-UCSB-IDS.csv'
    ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)

    print(df_days.head())
#     # tmax events stats, out as df
#     df_out = tmax_stats(df_days)

#     # merge to get countries
#     ghs_ids_df_out = ghs_ids_df.merge(df_out, on='ID_HDC_G0', how = 'inner')
#     # write it all out
#     ghs_ids_df_out.to_csv(DATA_IN+'chunk_stats+'+chunk+'.csv')

#     print(year, 'SAVED!')

In [50]:
def parallel_loop(function, file_list, cpu_num):
    """  EXPLAIN
    """ 
    start = time.time()
    pool = Pool(processes = cpu_num)
    pool.map(function, file_list)
    pool.close()

    end = time.time()
    print(end-start)